In [1]:
import torch
print(torch.__version__)

2.3.1


# Introducción

*Los **tensores** son una de las estructuras de datos más importantes dentro de PyTorch. Son arrays multidimensionales, similares a un `ndarray` de NumPy, pero con capacidades adicionales que los hacen muy útiles para el entrenamiento de modelos de Deep Learning.*

## Crear tensores

*Por default, PyTorch crea todos los tensores con el tipo de dato `torch.float32`. Sin embargo, hay muchos tipos de datos [disponibles](https://pytorch.org/docs/stable/tensors.html#data-types). Para definir un tipo de dato distinto, podemos utilizar el argumento `dtype` al crear el tensor.*

### Escalares

In [2]:
# Create a scalar
scalar = torch.tensor(7.0)
scalar

tensor(7.)

*Si quisieramos extraer el número entero de nuestro tensor tendríamos que utilizar el método `item()`.*

In [3]:
# Get a tensor back as an integer or float
print(f'Escalar: {scalar.item()}. Type: {type(scalar.item())}')

Escalar: 7.0. Type: <class 'float'>


### Vectores

*Los vectores son arrays unidimensionales. Utilizando el atributo `ndim` podemos validar que el tensor que creamos tiene una única dimensión. Podemos mirar con mas detalle las dimensiones del tensor con el atributo `shape` (notar que su dimensión tiene tamaño dos).*

In [4]:
# Create a vector
vector = torch.tensor([7, 7])
vector

tensor([7, 7])

In [5]:
# Dimensions
print(f'Number of dimensions: {vector.ndim}')

Number of dimensions: 1


In [6]:
print(f'Shape: {vector.shape}')

Shape: torch.Size([2])


### Matrices

*Las matrices son arrays **bi-dimensionales**, en donde la primera dimensión representa las filas y la segunda dimensión representa las columnas. El tamaño que tienen esas dimensiones hacen referencia la cantidad de filas y la cantidad de columnas de la matriz.*

In [7]:
MATRIX = torch.tensor([[7, 8], [8, 9]])
MATRIX

tensor([[7, 8],
        [8, 9]])

In [8]:
print(f'Number of dimensions: {MATRIX.ndim}')

Number of dimensions: 2


In [9]:
print(f'Shape: {MATRIX.shape}')

Shape: torch.Size([2, 2])


### Tensores

*Si bien los vectores y las matrices que vimos anteriormente también pueden ser llamados "tensores" (porque esta es la principal estructura de datos en PyTorch), los tensores propiamente dichos son arrays **n-dimensionales**. Por ejemplo, en el código debajo tenemos un tensor de tres dimensiones, aunque se puede extender fácilmente a más dimensiones.*

In [10]:
# Create a tensor
TENSOR = torch.tensor([[[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]]])
TENSOR

tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])

In [11]:
print(f'Number of dimensions: {TENSOR.ndim}')

Number of dimensions: 3


In [12]:
print(f'Shape: {TENSOR.shape}')

Shape: torch.Size([1, 3, 3])


*Para crear un tensor a partir de otro(s) tensor(es) como input(s), es necesario utilizar funciones como `stack()` o `vstack()`. Esto es así porque la clase `torch.tensor()` solo acepta objetos nativos de Python.*

*En este caso utilizamos la función `stack()` que nos permite concatenar tensores sobre una nueva dimensión. Si el argumento `dim = 0` entonces crea una primera nueva dimensión para concatenar los tensores (en nuestro caso, como tenemos tensores de dos dimensiones, crea una tercera dimensión).*

In [13]:
MATRIX_1 = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
MATRIX_2 = torch.tensor([[10, 11, 12], [13, 14, 15], [16, 17, 18]])

# The torch.tensor() class expects Python native data types, so in order to create a tensor from two tensors
TENSOR = torch.stack([MATRIX_1, MATRIX_2])
TENSOR

tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]]])

In [14]:
print(f'Number of dimensions: {TENSOR.ndim}')

Number of dimensions: 3


In [15]:
print(f'Shape: {TENSOR.shape}')

Shape: torch.Size([2, 3, 3])


### Tensores aleatorios

*Por qué queremos aprender a generar tensores de forma aleatoria? Porque los pesos de las redes neuronales se inicializan de manera aleatoria. Entonces, al comenzar el entrenamiento de nuestra red neuronal debemos pasarle un tensor que represente los pesos iniciales, el cual tenemos que poder generar de manera aleatoria.*

In [16]:
# Tensor de dos dimensiones de tamaño (3, 4)
random_tensor = torch.rand(size=(2, 3))
random_tensor

tensor([[0.9704, 0.4801, 0.2025],
        [0.8203, 0.9294, 0.6374]])

In [17]:
print(f'Dimensions: {random_tensor.ndim}')
print(f'Tamaños: {random_tensor.shape}')

Dimensions: 2
Tamaños: torch.Size([2, 3])


In [18]:
# Tensor de dos dimensiones de tamaño (3, 4)
random_tensor = torch.rand(size=(10, 2, 3))
random_tensor

tensor([[[0.2223, 0.7616, 0.5721],
         [0.9461, 0.2712, 0.2611]],

        [[0.7423, 0.2800, 0.5012],
         [0.0531, 0.8662, 0.2918]],

        [[0.4599, 0.1180, 0.6111],
         [0.6033, 0.8076, 0.2939]],

        [[0.7429, 0.4074, 0.8482],
         [0.0263, 0.0058, 0.5888]],

        [[0.8081, 0.4449, 0.8161],
         [0.6445, 0.8842, 0.3213]],

        [[0.6095, 0.7087, 0.1721],
         [0.2330, 0.0630, 0.8016]],

        [[0.9514, 0.5632, 0.6325],
         [0.3030, 0.5803, 0.4163]],

        [[0.5859, 0.3571, 0.6655],
         [0.4124, 0.1630, 0.3478]],

        [[0.6879, 0.2945, 0.1724],
         [0.8044, 0.7484, 0.5899]],

        [[0.2208, 0.5464, 0.5641],
         [0.4732, 0.1654, 0.0155]]])

In [19]:
print(f'Dimensions: {random_tensor.ndim}')
print(f'Tamaños: {random_tensor.shape}')

Dimensions: 3
Tamaños: torch.Size([10, 2, 3])


*Las imagenes se pueden representar como tensores de tres dimensiones, en donde la primer dimensión representa los canales de colores (RGB) y la segunda y tercer dimensión representa la cantidad de píxeles de la imágen. Por ejemplo, si tenemos una imagen a color de 1024x1024 y la convertimos en un tensor, entonces tendríamos un tensor de tres dimensiones con tamaño `torch.Size([3, 1024, 1024])`.*

In [20]:
random_image_tensor = torch.rand(size=(3, 1024, 1024))
random_image_tensor

tensor([[[0.3000, 0.8621, 0.2343,  ..., 0.0176, 0.0604, 0.2521],
         [0.8843, 0.7365, 0.6601,  ..., 0.6733, 0.5633, 0.8087],
         [0.4394, 0.8064, 0.3360,  ..., 0.0966, 0.0052, 0.2685],
         ...,
         [0.8810, 0.2448, 0.8114,  ..., 0.7054, 0.5087, 0.4212],
         [0.8028, 0.9815, 0.8673,  ..., 0.8679, 0.5931, 0.5669],
         [0.0585, 0.9454, 0.0104,  ..., 0.7831, 0.6046, 0.3545]],

        [[0.4583, 0.6332, 0.5540,  ..., 0.9738, 0.5965, 0.5811],
         [0.6091, 0.8477, 0.4484,  ..., 0.2950, 0.0983, 0.6595],
         [0.0158, 0.0589, 0.7507,  ..., 0.6091, 0.1694, 0.0061],
         ...,
         [0.0488, 0.5793, 0.7349,  ..., 0.7093, 0.2071, 0.5944],
         [0.8324, 0.0050, 0.6985,  ..., 0.3546, 0.5413, 0.3017],
         [0.3302, 0.7142, 0.0639,  ..., 0.6703, 0.0089, 0.1508]],

        [[0.9928, 0.4587, 0.9960,  ..., 0.1059, 0.3823, 0.7579],
         [0.9704, 0.4953, 0.1673,  ..., 0.9257, 0.3586, 0.0159],
         [0.3927, 0.4262, 0.9744,  ..., 0.5333, 0.6658, 0.

In [21]:
print(f'Dimensiones: {random_image_tensor.ndim}')
print(f'Shape: {random_image_tensor.shape}')

Dimensiones: 3
Shape: torch.Size([3, 1024, 1024])


### Unos y ceros

*En algunas ocaciones nos gustaría tener tensores que solo tienen ceros o unos. Los podemos crear con las funciones `torch.zeros()` o `torch.ones()`.*

In [22]:
zeros = torch.zeros(size=(2, 3, 4))
zeros

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [23]:
ones = torch.ones(size=(2, 3, 4))
ones

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

### Tensor-like objects

In [24]:
range_tensor = torch.arange(start=0, end=1000, step=36)
range_tensor

tensor([  0,  36,  72, 108, 144, 180, 216, 252, 288, 324, 360, 396, 432, 468,
        504, 540, 576, 612, 648, 684, 720, 756, 792, 828, 864, 900, 936, 972])

In [25]:
print(f'Dimension: {range_tensor.ndim}')
print(f'Shape: {range_tensor.shape}')
print(f'Dtype: {range_tensor.dtype}')

Dimension: 1
Shape: torch.Size([28])
Dtype: torch.int64


*Cuando decimos que vamos a crear o que creamos un objeto que es "tensor-like", nos referimos a crear un nuevo tensor que tenga la misma dimensión, tamaño y tipo de dato que otro tensor ya existente, pero con otros valores.*

*Por ejemplo, podemos creamos un nuevo tensor a partir del tensor `range_tensor` que tenga la misma dimensión, tamaño y tipo de dato, pero que la información que contiene sea distinta (solo tenga ceros).*

In [26]:
zeros = torch.zeros_like(input=range_tensor)
zeros

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [27]:
print(f'Dimension: {zeros.ndim}')
print(f'Shape: {zeros.shape}')
print(f'Dtype: {zeros.dtype}')

Dimension: 1
Shape: torch.Size([28])
Dtype: torch.int64
